In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
a=['../data/energy-y1.csv',
   '../data/mushroom.csv',
   '../data/pima.csv','../data/magic.csv',
   '../data/titanic.csv','../data/wine.csv',
   '../data/breast-cancer-wisc-diag_train.csv','../data/credit-approval.csv',
   '../data/spambase_train.csv','../data/statlog-image.csv',
   '../data/miniboone.csv','../data/semeion.csv',
    '../data/twonorm.csv','../data/planning.csv','../data/connect-4.csv']


In [3]:
def self_training(Z_labelled_,O_labelled_,Z_unlabelled_,sample_rate_unlabelled,Z_test,O_test):
    lr = LogisticRegression(C=1.)
    #lr=SVC(kernel='rbf')
    #lr=KNeighborsClassifier(n_neighbors=4)
    Z_labelled=Z_labelled_
    O_labelled=O_labelled_
    Z_unlabelled=Z_unlabelled_
    #required unlabelled data to move to labelled
    len_labelled_data=sample_rate_unlabelled*len(Z_unlabelled)
    i=0
    confidence=0.90
    while(1):
        lr.fit(Z_labelled,O_labelled)
        if(len_labelled_data >= len(Z_unlabelled) or i>20):
            break
        #predict probability of the unlabelled class
        temp=lr.predict_proba(Z_unlabelled)
        #index if confidence in class 1 is greater than 0.95
        temp_index_class1=np.asarray(np.where(temp[:,0] >confidence))
        if len(temp_index_class1[0,:]):
            
            label=lr.predict([Z_unlabelled[temp_index_class1[0,0],:]])
            Z_labelled=np.concatenate((Z_labelled, Z_unlabelled[temp_index_class1[0,:],:]), axis=0)  
            O_labelled=np.concatenate((O_labelled, np.full((len(temp_index_class1[0,:])), label)), axis=0) 
        #index if confidence in class 2 is greater than 0.95
        temp_index_class2=np.asarray(np.where(temp[:,1] >confidence))
        #print(temp_index_class2.shape,len(temp_index_class1))
        if len(temp_index_class2[0,:]):
            label=lr.predict([Z_unlabelled[temp_index_class2[0,0],:]])
            Z_labelled=np.concatenate((Z_labelled, Z_unlabelled[temp_index_class2[0,:],:]), axis=0)  
            O_labelled=np.concatenate((O_labelled, np.full((len(temp_index_class2[0,:])), label)), axis=0)  
        #Now clean the unlabelled data from the labbelled data
        Z_2=np.concatenate((temp_index_class1[0,:], temp_index_class2[0,:]), axis=0) 
        Z_unlabelled=np.delete(Z_unlabelled,Z_2,0)
        i=i+1

    return (1-lr.score(Z_test,O_test))*100

In [4]:
for k in range(0,14):
    data=np.genfromtxt(a[k], delimiter=',')
    print("Dataset:",a[k][8:])
    dataj=[]
    dataj=np.array_split(data,5)
    X_labelled=[]
    X_unlabelled=[]
    Y_labelled=[]
    for i in range(0,5):
        random_data = np.random.randint(len(dataj[i]), size=int(0.15*len(dataj[i])))
        X_labelled.append(dataj[i][random_data,:-1])
        Y_labelled.append(dataj[i][random_data,-1])
        X_unlabelled.append(np.delete(dataj[i][:,:-1],random_data,0))
        #print(random_data.shape,dataj[i].shape,X_labelled[i].shape,X_unlabelled[i].shape,Y_labelled[i].shape)
    error=0
    error1=0
    for i in range(0,5):
        #the testing array
        Z_test=X_labelled[i]
        O_test=Y_labelled[i]
        first=1
        for j in range(0,5):
            #the training array
            if(i!=j):
                if(first==1):
                    Z_labelled=X_labelled[j]
                    O_labelled=Y_labelled[j]
                    Z_unlabelled=X_unlabelled[j]
                    first=0
                else:
                    Z_labelled=np.concatenate((Z_labelled,X_labelled[j]),axis=0)
                    O_labelled=np.concatenate((O_labelled,Y_labelled[j]),axis=0)
                    Z_unlabelled=np.concatenate((Z_unlabelled,X_unlabelled[j]),axis=0)
        #print(Z_test.shape,O_test.shape,Z_labelled.shape,O_labelled.shape,Z_unlabelled.shape)
        error=error+self_training(Z_labelled,O_labelled,Z_unlabelled,0,Z_test,O_test)
        error1=error1+self_training(Z_labelled,O_labelled,Z_unlabelled,1,Z_test,O_test)
    print("5 fold validation error by using self training on unlabelled data",error/5) 
     
    print("5 fold validation error without labelling the training data",error1/5)
    
    

Dataset: energy-y1.csv
5 fold validation error by using self training on unlabelled data 19.4466403162
5 fold validation error without labelling the training data 19.4466403162
Dataset: mushroom.csv
5 fold validation error by using self training on unlabelled data 22.0576131687
5 fold validation error without labelling the training data 22.0576131687
Dataset: pima.csv
5 fold validation error by using self training on unlabelled data 29.209486166
5 fold validation error without labelling the training data 28.3399209486
Dataset: magic.csv
5 fold validation error by using self training on unlabelled data 28.4210526316
5 fold validation error without labelling the training data 28.0701754386
Dataset: titanic.csv
5 fold validation error by using self training on unlabelled data 19.696969697
5 fold validation error without labelling the training data 19.696969697
Dataset: wine.csv
5 fold validation error by using self training on unlabelled data 12.0
5 fold validation error without labelling